Cell for working not locally

Make git clone if we aren't in the right directory

In [1]:
import subprocess
import os

repo_path = "YAGPT"
branch = "dev"

if not os.path.isdir(repo_path):
    subprocess.run(["git", "clone", "https://github.com/RodionfromHSE/YAGPT.git"])
    
os.chdir(repo_path)
subprocess.run(["git", "checkout", branch])

Cloning into 'YAGPT'...


branch 'dev' set up to track 'origin/dev'.


Switched to a new branch 'dev'


CompletedProcess(args=['git', 'checkout', 'dev'], returncode=0)

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Windows users

In [9]:
from omegaconf import OmegaConf

params = OmegaConf.load("config.yaml")

In [2]:
%%capture
!pip install transformers datasets huggingface_hub

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from datasets import load_dataset

dataset = load_dataset(params.args.dataset)

Using custom data configuration under-tree--prepared-yagpt-57938d8c9614b5cb
Found cached dataset parquet (/home/alexw/.cache/huggingface/datasets/under-tree___parquet/under-tree--prepared-yagpt-57938d8c9614b5cb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from pprint import pprint
print('len', len(dataset))
pprint(dataset['train'][0])
pprint(dataset['train'][-7])

len 2
{'text': 'history: \n'
         'speaker1: как земные географы относятся к идее освоения и '
         'колонизации луны и марса?\tspeaker2: к планам освоения луны отношусь '
         'с большой осторожностью. \n'
         'есть одна грань, о которой у нас почти не говорят. на луне очень '
         'страшная пыль. дело в том, что эта пыль размером в 2-3 микрона. '
         'каждая пылинка имеет зазубренные концы. попадая в легкие, такая пыль '
         'вызывает страшную болезнь – лунную сенную лихорадку. все 15 '
         'астронавтов из экспедиции космического корабля "аполлон", среди '
         'которых были и те, которые не выходили на поверхность луны, '
         'переболели этой болезнью. скафандры до пояса были покрыты пылью. она '
         'попадала на всех участников экспедиции. пока ученые не знают, как '
         'избавиться от лунной пыли.\n'
         'проблема с освоением марса, даже если хватит денег туда долететь и '
         'там поработать, связана с возможностью 

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer


checkpoint = params.model.model_name
tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token='<|pad|>')

special_tokens = {'additional_special_tokens': list(params.args.special_tokens)}
tokenizer.add_special_tokens(special_tokens)

model = AutoModelForCausalLM.from_pretrained(checkpoint)
model.resize_token_embeddings(len(tokenizer))
None

2023-04-17 16:35:21.086587: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-17 16:35:21.086610: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-17 16:35:22.890836: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-17 16:35:22.937339: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-17 16:35:22.939715: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic l

In [7]:
import multiprocessing

num_proc = multiprocessing.cpu_count()

In [8]:
def tokenize(elem):
    inputs = tokenizer(elem['text'])
    return inputs

data_token = dataset.map(tokenize, batched=True, remove_columns=dataset['train'].column_names, num_proc=num_proc)

In [26]:
data_prep

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 131870817
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 14603381
    })
})

In [32]:
block_size = params.args.max_len

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    #     customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

data_prep = data_token.map(group_texts, batched=True, num_proc=num_proc)

In [33]:
for (key, value) in data_prep['train'][0].items():
  print(f'key {key}, value len {len(value)}, value type {type(value)}')

key input_ids, value len 128, value type <class 'list'>
key attention_mask, value len 128, value type <class 'list'>
key labels, value len 128, value type <class 'list'>


In [18]:
del training_args

In [41]:
!nvidia-smi

Sun Apr  9 18:01:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    27W /  70W |  15101MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [34]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

10

In [46]:
!nvidia-smi

Sun Apr  9 18:03:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    27W /  70W |  15101MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [39]:
from transformers import Trainer, TrainingArguments

batch_size = params.args.batch_size
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size
)

trainer = Trainer(
    model=model,                         
    args=training_args,
    train_dataset=data_prep['train'],
    eval_dataset=data_prep['test']
)

In [ ]:
trainer.train()

/home/alexw/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1029863
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 386199
  Number of trainable parameters = 355881984
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alexwortega. Use `wandb login --relogin` to force relogin


Automatic ClearML logging enabled.


ClearML Task: created new task id=8af22ab3a2dd4a388874671c212c8abc
2023-04-17 17:06:27,285 - clearml.Repository Detection - WARNING - Password protected Jupyter Notebook server was found! Add `sdk.development.jupyter_server_password=<jupyter_password>` to ~/clearml.conf
2023-04-17 17:06:27,510 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/32528037304a461d86dada2b2c2dd3e3/experiments/8af22ab3a2dd4a388874671c212c8abc/output/log


ClearML Task has been initialized.


Epoch,Training Loss,Validation Loss


In [ ]:
import math

eval_result = trainer.evaluate() # returns a dict with the evaluation metrics, cross-entropy loss and perplexity
pprint(eval_result)